In [ ]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# # install oct2py that compatible with colab
import os

from pkg_resources import get_distribution

os.system(f"pip install -qq"
          f" ipykernel=={get_distribution('ipykernel').version}"
          f" ipython=={get_distribution('ipython').version}"
          f" tornado=={get_distribution('tornado').version}"
          f" oct2py"
          )

# install packages
!pip install -qq matpower matpowercaseframes

In [ ]:
import oct2py

import matpower

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.6.0
matpower version: 7.1.0.2.1.4


In [ ]:
import os

import numpy as np
import pandas as pd
from matpowercaseframes import CaseFrames
from matpowercaseframes.constants import COLUMNS

from matpower import path_matpower, start_instance

## Inspect data

In [ ]:
path = os.path.join(path_matpower, 'data/case9.m')

In [ ]:
cf = CaseFrames(path)
cf.branch

,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,BR_STATUS,ANGMIN,ANGMAX
1,1,4,0.0000,0.0576,0.000,250,250,250,0,0,1,-360,360
2,4,5,0.0170,0.0920,0.158,250,250,250,0,0,1,-360,360
3,5,6,0.0390,0.1700,0.358,150,150,150,0,0,1,-360,360
4,3,6,0.0000,0.0586,0.000,300,300,300,0,0,1,-360,360
5,6,7,0.0119,0.1008,0.209,150,150,150,0,0,1,-360,360
6,7,8,0.0085,0.0720,0.149,250,250,250,0,0,1,-360,360
7,8,2,0.0000,0.0625,0.000,250,250,250,0,0,1,-360,360
8,8,9,0.0320,0.1610,0.306,250,250,250,0,0,1,-360,360
9,9,4,0.0100,0.0850,0.176,250,250,250,0,0,1,-360,360


In [ ]:
m = start_instance()
sol = m.loadcase('case9', verbose=False)
pd.DataFrame(sol['branch'], columns=cf.branch.columns)

,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,BR_STATUS,ANGMIN,ANGMAX
0,1.0,4.0,0.0000,0.0576,0.000,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
1,4.0,5.0,0.0170,0.0920,0.158,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
2,5.0,6.0,0.0390,0.1700,0.358,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0
3,3.0,6.0,0.0000,0.0586,0.000,300.0,300.0,300.0,0.0,0.0,1.0,-360.0,360.0
4,6.0,7.0,0.0119,0.1008,0.209,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0
5,7.0,8.0,0.0085,0.0720,0.149,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
6,8.0,2.0,0.0000,0.0625,0.000,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
7,8.0,9.0,0.0320,0.1610,0.306,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
8,9.0,4.0,0.0100,0.0850,0.176,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0


## Run base

In [ ]:
mpc = m.runpf(sol, verbose=True)


MATPOWER Version 8.0b1, 22-Dec-2022 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.84 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transform

## Copy and editing branch

In [ ]:
new_branch = sol['branch'][2].copy()  # copy third branch, must use .copy()
new_branch[COLUMNS['branch'].index('T_BUS')] = 8  # connect to bus 8
sol['branch'] = np.vstack((sol['branch'], new_branch))
pd.DataFrame(sol['branch'], columns=cf.branch.columns)

,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,BR_STATUS,ANGMIN,ANGMAX
0,1.0,4.0,0.0000,0.0576,0.000,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
1,4.0,5.0,0.0170,0.0920,0.158,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
2,5.0,6.0,0.0390,0.1700,0.358,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0
3,3.0,6.0,0.0000,0.0586,0.000,300.0,300.0,300.0,0.0,0.0,1.0,-360.0,360.0
4,6.0,7.0,0.0119,0.1008,0.209,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0
5,7.0,8.0,0.0085,0.0720,0.149,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
6,8.0,2.0,0.0000,0.0625,0.000,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
7,8.0,9.0,0.0320,0.1610,0.306,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
8,9.0,4.0,0.0100,0.0850,0.176,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0
9,5.0,8.0,0.0390,0.1700,0.358,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0


In [ ]:
mpc = m.runpf(sol, verbose=True)


MATPOWER Version 8.0b1, 22-Dec-2022 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.29 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    318.5             -24.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches          10     Losses (I^2 * Z)         3.45             42.55
Transform

In [ ]:
pd.DataFrame(mpc['branch'], columns=COLUMNS['branch'][:17])

,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,BR_STATUS,ANGMIN,ANGMAX,PF,QF,PT,QT
0,1.0,4.0,0.0000,0.0576,0.000,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,70.454094,10.603728,-70.454094,-7.900417
1,4.0,5.0,0.0170,0.0920,0.158,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,10.809598,-9.436561,-10.790898,-7.369035
2,5.0,6.0,0.0390,0.1700,0.358,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0,-40.582184,-11.386880,41.204848,-24.356641
3,3.0,6.0,0.0000,0.0586,0.000,300.0,300.0,300.0,0.0,0.0,1.0,-360.0,360.0,85.000000,-22.326555,-85.000000,26.634426
4,6.0,7.0,0.0119,0.1008,0.209,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0,43.795152,-2.277785,-43.574748,-18.134364
5,7.0,8.0,0.0085,0.0720,0.149,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,-56.425252,-16.865636,56.688913,3.234035
6,8.0,2.0,0.0000,0.0625,0.000,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,-163.000000,28.950311,163.000000,-13.043626
7,8.0,9.0,0.0320,0.1610,0.306,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,67.117183,-7.485969,-65.754557,-17.686006
8,9.0,4.0,0.0100,0.0850,0.176,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,-59.245443,-32.313994,59.644496,17.336977
9,5.0,8.0,0.0390,0.1700,0.358,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0,-38.626918,-11.244085,39.193903,-24.698376
